Notre objectif est de réaliser un code permettant de classifier des données entre spam et non spam

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import re

In [2]:
data=pd.read_csv("D://codes//ia//data//spam.csv", encoding_errors="replace")
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will �_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data['v1'] = encoder.fit_transform(data['v1'])

In [5]:
data.describe()

,v1
count,5572.000000
mean,0.134063
std,0.340751
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   int32 
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: int32(1), object(4)
memory usage: 196.0+ KB


In [7]:
data.rename(columns = {'v1': 'target', 'v2': 'text'}, inplace = True)

In [8]:
data["target"].unique()

array([0, 1])

In [9]:
data.isnull().sum()

target           0
text             0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [10]:
data.duplicated().sum()

403

In [11]:
data = data.drop_duplicates(keep = 'first')

informations à récolter pour chaque message:
-nombre de phrases
-présence ou absence de caractères spéciaux
-nombre de caractères
-présence ou absence de nombres
-présence ou absence de numéros de téléphone
-nombre de mots par messages
-présence de phrases exclamatives 
-présence de phrases interrogatives
-présence de séquences de caractères en majuscules


après analyse, nous sortirons les mots les plus communs au niveau des deux catégories de texte pour créer deux nouvelles colomnes

In [12]:
#création d'un second data set associé au premier pour récupérer les informations essentielles
df= pd.DataFrame()
df

""


In [ ]:
import re

# Crée toutes les colonnes nécessaires dans df
for col in range(1, 8):
    df[col] = 0
    
for i in data.index:
    if data.iloc[i,2]or data.iloc[i,3] or  data.iloc[i,4]:
        df.loc[i,1]=1
    else:
        df.loc[i,1]=0

# Parcourir les lignes
for i in data.index:
    if df.iloc[i, 0] == 1:
        # Concaténation des colonnes 1 à 3 (en chaînes de caractères)
        combined =  str(data.iloc[i, 1]) + str(data.iloc[i, 2]) + str(data.iloc[i, 3])
        data.loc[i, 4] = combined

        # Longueur du texte combiné
        df.loc[i, 1] = len(combined)

        # Caractères non encodés (à définir clairement, ici exemple : caractères spéciaux non ASCII)
        if re.search(r'[^\x00-\x7F]', combined):
            df.loc[i, 2] = 1

        # Contient un nombre
        if re.search(r'\d', combined):
            df.loc[i, 3] = 1

        # Contient un numéro de téléphone simple 
        if re.search(r'\b\d{8,15}\b', combined):
            df.loc[i, 4] = 1

        # Contient une adresse email
        if re.search(r'\b[\w\.-]+@[\w\.-]+\.\w{2,4}\b', combined):
            df.loc[i, 5] = 1

        # Contient une séquence de MAJUSCULES de 2 lettres ou plus
        maj_seq = re.findall(r'[A-Z]{2,}', combined)
        df.loc[i, 6] = len(maj_seq)


        df.loc[i, 6]=data.iloc[i, 6]




In [ ]:
df


In [13]:
data.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace = True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7184\1191422558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace = True)


In [14]:
data

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will �_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [15]:
#debut de l'analyse de nos données

In [ ]:
print(df.info())



In [ ]:
# Statistiques descriptives sur les données numériques
df.describe()

In [ ]:
sns.pairplot(df,height=1.5)

In [ ]:
sns.heatmap(df.corr(),annot=False )